In [1]:

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['trainpredict', 'testpredict']


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X = pd.read_csv("../input/trainpredict/train.csv")
X_1 = pd.read_csv("../input/testpredict/test.csv")

In [4]:
train = X
test = X_1
train.isnull().sum(axis=0)
test.isnull().sum(axis = 0)

ID                  0
datetime            0
siteid         370776
offerid             0
category            0
merchant            0
countrycode         0
browserid      221906
devid          704619
dtype: int64

In [5]:
train = train.replace('NaN', np.NAN)
test = test.replace('NaN',np.NAN)
train['siteid'].fillna(-111, inplace = True)
test['siteid'].fillna(-111, inplace = True)
train['browserid'].fillna("x", inplace = True)
test['browserid'].fillna("x", inplace = True)
train['devid'].fillna("xx", inplace = True)
test['devid'].fillna("xx", inplace = True)
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,e,Firefox,xx,0
1,IDmMSxHur,2017-01-18 17:50:53,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0
2,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,a,Edge,xx,0
3,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0
4,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0


In [6]:
train['datetime'] = pd.to_datetime(train['datetime'])
train['day'] = train['datetime'].dt.weekday
train['hour'] = train['datetime'].dt.hour

test['datetime'] = pd.to_datetime(test['datetime'])
test['day'] = test['datetime'].dt.weekday
test['hour'] = test['datetime'].dt.hour

train['hour1'] = train['hour']%4
test['hour1'] = test['hour']%4

train = train.drop(columns=['ID', 'datetime', 'hour',]) #'siteid' ,]) #,'offerid'])
test = test.drop(columns=['datetime', 'hour', ])#'siteid', ]) #'offerid'])

train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,day,hour1
0,4709696.0,887235,17714,20301556,e,Firefox,xx,0,5,1
1,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0,2,1
2,98480.0,518539,25085,2050923,a,Edge,xx,0,2,0
3,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0,1,2
4,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0,5,0


In [7]:

train = train.astype('object')
test = test.astype('object')

In [8]:
from sklearn.preprocessing import LabelEncoder

#cols = ['siteid','offerid','category','merchant', 'countrycode','browserid','devid']
for col in test.columns[1:]:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))

In [9]:
train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,day,hour1
0,128865,784773,48,127,4,2,3,0,5,1
1,142053,157563,59,65,1,8,0,0,2,1
2,2618,458279,69,15,0,1,3,0,2,0
3,243406,345067,117,507,2,2,1,0,1,2
4,154278,417948,36,276,3,8,0,0,5,0


In [10]:
from sklearn.model_selection import train_test_split

Y_t = train['click'].astype('float')
X_t = train.drop(columns=['click'])
#Y_t = Y_t.as_type('float')
X_train, X_test, y_train, y_test = train_test_split(X_t, Y_t, test_size=0.1, random_state=69)

**Naive Bayesian Model**

In [11]:

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

from sklearn.metrics import confusion_matrix, roc_auc_score
predicted = model.predict(X_test)
print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))

[[600257 569672]
 [ 22222  21630]]
0.5031606644202486


In [12]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB().fit(X_train, y_train)
predicted = model.predict(X_test)

from sklearn.metrics import confusion_matrix, roc_auc_score

print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))

[[1169929       0]
 [  43852       0]]
0.5


In [13]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB().fit(X_train, y_train)
predicted = model.predict(X_test)

from sklearn.metrics import confusion_matrix, roc_auc_score

print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))

[[1169929       0]
 [  43838      14]]
0.5001596278390952


**Decision Tree**

In [14]:

from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier()
clf_gini.fit(X_train, y_train)
predicted = clf_gini.predict(X_test)
from sklearn.metrics import confusion_matrix, roc_auc_score

print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))

[[1147782   22147]
 [  20119   23733]]
0.7611382889391259


In [15]:
from sklearn.tree import DecisionTreeClassifier
clf_ent = DecisionTreeClassifier(criterion = "entropy")
clf_ent.fit(X_train, y_train)
predicted = clf_ent.predict(X_test)
from sklearn.metrics import confusion_matrix, roc_auc_score

print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))

preds = clf_ent.predict(test[test.columns[1:]])
sub2 = pd.DataFrame({'ID':test['ID'], 'click':preds})
sub2.to_csv('submission_dt.csv', index=False)

[[1148461   21468]
 [  20075   23777]]
0.7619301649833284


**Random Forest**

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 50, max_depth = 12, random_state = 69)
rf.fit(X_train, y_train)
predicted = rf.predict(X_test)
from sklearn.metrics import confusion_matrix, roc_auc_score

print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))

preds = rf.predict(test[test.columns[1:]])
sub1 = pd.DataFrame({'ID':test['ID'], 'click':preds})
sub1.to_csv('submission_random.csv', index=False)

[[1158744   11185]
 [  17116   26736]]
0.8000633601069796


In [17]:
ls

__notebook_source__.ipynb  submission_dt.csv  submission_random.csv
